In [1]:
import torch

In [4]:
torch.__version__

'1.1.0'

In [5]:
torch.cuda.is_available()

False

In [6]:

from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [28]:
parser = argparse.ArgumentParser(description='VAE MNIST Example')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='enables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')

batch_size=128
epochs=10
log_interval=100
seed=1

cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

In [29]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


In [31]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


In [32]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [35]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


In [36]:
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

In [38]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png')


Train Epoch: 1 [0/60000 (0%)]	Loss: 118.621101
Train Epoch: 1 [12800/60000 (21%)]	Loss: 110.434624
Train Epoch: 1 [25600/60000 (43%)]	Loss: 113.340065
Train Epoch: 1 [38400/60000 (64%)]	Loss: 116.755386
Train Epoch: 1 [51200/60000 (85%)]	Loss: 108.267685
====> Epoch: 1 Average loss: 114.6074
====> Test set loss: 111.8555
Train Epoch: 2 [0/60000 (0%)]	Loss: 111.358978
Train Epoch: 2 [12800/60000 (21%)]	Loss: 107.061127
Train Epoch: 2 [25600/60000 (43%)]	Loss: 114.494041
Train Epoch: 2 [38400/60000 (64%)]	Loss: 106.754814
Train Epoch: 2 [51200/60000 (85%)]	Loss: 114.545807
====> Epoch: 2 Average loss: 111.5671
====> Test set loss: 109.5794
Train Epoch: 3 [0/60000 (0%)]	Loss: 104.894104
Train Epoch: 3 [12800/60000 (21%)]	Loss: 107.635330
Train Epoch: 3 [25600/60000 (43%)]	Loss: 107.619278
Train Epoch: 3 [38400/60000 (64%)]	Loss: 108.788017
Train Epoch: 3 [51200/60000 (85%)]	Loss: 109.550995
====> Epoch: 3 Average loss: 109.7161
====> Test set loss: 108.2817
Train Epoch: 4 [0/60000 (0%)]	L

In [41]:
model.state_dict()['fc1.weight']

tensor([[ 0.0002, -0.0345, -0.0009,  ..., -0.0027, -0.0115,  0.0214],
        [ 0.0353,  0.0217, -0.0138,  ...,  0.0175, -0.0148, -0.0144],
        [-0.0323,  0.0176,  0.0317,  ...,  0.0139,  0.0229,  0.0055],
        ...,
        [ 0.0230,  0.0242, -0.0216,  ..., -0.0272, -0.0094,  0.0178],
        [-0.0018, -0.0211, -0.0226,  ...,  0.0227, -0.0001,  0.0340],
        [-0.0074,  0.0116, -0.0033,  ...,  0.0298, -0.0254,  0.0352]])

In [42]:
optimizer.state_dict()

{'state': {139626076512192: {'step': 5628,
   'exp_avg': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]]),
   'exp_avg_sq': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]])},
  139626076509096: {'step': 5628,
   'exp_avg': tensor([ 2.7874e+00,  5.6052e-45, -1.0496e+00,  4.8892e-02,  2.8884e+00,
            2.6043e+00,  2.1176e-01, -1.9953e+00,  5.6052e-45, -1.5005e+00,
           -1.3496e-01,  5.6052e-45,  1.2239e+00,  5.6052e-45,  5.6052e-45,
            5.6052e-45,  1.1401e+00, -1.3474e+00,  3.9741e+00,  5.6052e-45,
            5.6052e-45,  5.605

In [45]:
torch.save({ 
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, "model")

In [47]:
d =torch.load("model")

In [49]:
d.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict'])